# 1. download data

In [ ]:
import os
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
client = Client()

# define data unit and time interval
symbol = 'ETHUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE

# define start & end date
start_date = datetime.now() - timedelta(days=365 * 3)  # before 365 days
end_date = datetime.now()  # current

# create folder data storage
data_folder = 'ETHUSDT'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)


# function save daily data to single csv
def get_historical_data(symbol, interval, start_date, end_date):
    # convert start & end datetime to ms
    start_time = int(start_date.timestamp() * 1000)
    end_time = int(end_date.timestamp() * 1000)

    # extract
    klines = client.get_historical_klines(symbol, interval, start_time, end_time)

    # convert to DataFrame
    df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                       'close_time', 'quote_asset_volume', 'number_of_trades',
                                       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    # convert ts to dt
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # keep only relavant
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

    return df


# download and save data 
def download_data_by_day(symbol, interval, start_date, end_date, data_folder):
    current_date = start_date
    while current_date < end_date:
        next_date = current_date + timedelta(days=1)
        print(f"Downloading data from {current_date.strftime('%Y-%m-%d')} to {next_date.strftime('%Y-%m-%d')}")

        # get daily data
        daily_data = get_historical_data(symbol, interval, current_date, next_date)

        # save to csv
        filename = os.path.join(data_folder, f"ethusdt_{current_date.strftime('%Y-%m-%d')}.csv")
        daily_data.to_csv(filename, index=False)

        # update dt
        current_date = next_date


# data saved in 'data' folder
download_data_by_day(symbol, interval, start_date, end_date, data_folder)


# merge data csv
def merge_csv_files(data_folder):
    import glob

    # get file names
    all_files = glob.glob(os.path.join(data_folder, "*.csv"))

    # read and merge
    df_list = [pd.read_csv(file) for file in all_files]
    merged_df = pd.concat(df_list, ignore_index=True)

    return merged_df


# merge all csv in 'data' folder
merged_data = merge_csv_files(data_folder)

# sort by ts
merged_data = merged_data.sort_values(by='timestamp')

# save to data.csv
merged_data.to_csv('data.csv', index=False)

print("data is merged and sorted by time，saved in 'data.csv'")


# 2. strategy simulations

In [5]:
import pandas as pd
import numpy as np
import itertools
import os

def eth_grid_trading_strategy(file_path, output_path, nrows=None, min_price=1800, max_price=3600, grid_count=100,
                              initial_price=2700, time_interval=3,max_size =10,grid_unit_size = 0.1,free_rate = 0.0005):
    """
    grid trading strategies

    parameters:
        file_path (str): input csv file path
        output_path (str): output csv file path 
        nrows (int, optional): number of rows to read, default=all
        min_price (float): minimum price, default=1800
        max_price (float): maximum price, default=3600
        grid_count (int): number of grids, defailt=100
        initial_price (float): initial price, default=2700。
        time_interval (int): time interval for trade decisions, default=3 min
        max_size : maximum trade amount
        grid_unit_size : amount in every grid trade
    return:
        None
    """

    # read input data
    df = pd.read_csv(file_path, nrows=nrows)

    # generate grid points
    grid_points = np.geomspace(min_price, max_price, grid_count)

    position_2700 = np.searchsorted(grid_points, initial_price, side='right')

    # strategy parameters
    # max_position = 10  # maximum trade amount (ETH)
    # grid_size = 0.1  # ETH amount in every grid

    # new cols for recording decisions and trade amount
    df['Decision'] = ''
    df['Amount'] = np.nan
    df['Position'] = 0.0  
    df['money'] = 0.0  

    # every time_interval in minute
    df = df.iloc[::time_interval, :].reset_index(drop=True)

    # calculate the dif btw each price and the grid points and decide
    df['grid'] = np.searchsorted(grid_points, df['close'], side='right')

    position = 0
    money = 0
    # vectorize trading logic
    for i, row in df.iterrows():
        price = row['close']
        cross_grid = abs(row['grid'] - position_2700) + 1

        if price <= min_price or price >= max_price:
            df.at[i, 'Position'] = position
            df.at[i, 'money'] = money
            continue

        # buy
        if price < initial_price:
            grid_size = max(1, min(max_size, cross_grid))
            amount = grid_size * grid_unit_size
            df.at[i, 'Decision'] = 'Buy'
            df.at[i, 'Amount'] = amount
            position += amount
            money -= amount * price
            money -= amount * price *free_rate

        # sell
        elif price > initial_price:
            grid_size = max(1, min(max_size, cross_grid))
            amount = grid_size * grid_unit_size
            amount = min(position , amount)
            if amount > 0:
                df.at[i, 'Decision'] = 'Sell'
                df.at[i, 'Amount'] = grid_size
                position -= amount
                money += amount * price
                money -= amount * price * free_rate


        # update position
        df.at[i, 'Position'] = position
        df.at[i, 'money'] = money


    # delete misc
    df.drop(columns=['grid'], inplace=True)

    # output csv
    df.to_csv(output_path, index=False)
    print(f"executed strategies, saved results in {output_path}")


def grid_search_eth_strategy(file_path, output_folder, nrows=None, min_price=1800, max_price=3600,
                             grid_densities=[80, 100, 120], initial_prices=[2400, 2700, 3000],
                             time_intervals=[1, 5, 15], max_size=10, grid_unit_size=0.1, free_rate=0.0005):
    """
    grid search over trade stratigies

    parameters:
        file_path (str): input csv file path
        output_folder (str): output csv file path 
        nrows (int, optional): number of rows to read, default=all
        min_price (float): minimum price, default=1800
        max_price (float): maximum price, default=3600
        grid_densities (list of int): grid density list 
        initial_prices (list of float): initial price list
        time_intervals (list of int): time interval list
        max_size (int): maximum trade amount
        grid_unit_size (float): amount in every grid trade
        free_rate (float): processing fee
    return:
        None
    """
    # create output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # generate all combinations
    param_combinations = list(itertools.product(grid_densities, initial_prices, time_intervals))

    # search thru combinations and call func eth_grid_trading_strategy
    for grid_count, initial_price, time_interval in param_combinations:
        output_file = os.path.join(output_folder, f"grid{grid_count}_price{initial_price}_interval{time_interval}.csv")
        eth_grid_trading_strategy(file_path, output_file, nrows=nrows, min_price=min_price, max_price=max_price,
                                  grid_count=grid_count, initial_price=initial_price, time_interval=time_interval,
                                  max_size=max_size, grid_unit_size=grid_unit_size, free_rate=free_rate)


# grid search
grid_search_eth_strategy(
    file_path='data.csv',
    output_folder='simulations',  # output files save in 'simulation' folder
    nrows=100000,
    min_price=1800,
    max_price=3600,
    grid_densities=[80, 100, 120],
    initial_prices=[2400, 2700, 3000, 3300],
    time_intervals=[1, 3, 5, 15],
    grid_unit_size=0.1,
    max_size=10,
    free_rate=0.0005
)

executed strategies, saved results in simulations/grid80_price2400_interval1.csv
executed strategies, saved results in simulations/grid80_price2400_interval3.csv
executed strategies, saved results in simulations/grid80_price2400_interval5.csv
executed strategies, saved results in simulations/grid80_price2400_interval15.csv
executed strategies, saved results in simulations/grid80_price2700_interval1.csv
executed strategies, saved results in simulations/grid80_price2700_interval3.csv
executed strategies, saved results in simulations/grid80_price2700_interval5.csv
executed strategies, saved results in simulations/grid80_price2700_interval15.csv
executed strategies, saved results in simulations/grid80_price3000_interval1.csv
executed strategies, saved results in simulations/grid80_price3000_interval3.csv
executed strategies, saved results in simulations/grid80_price3000_interval5.csv
executed strategies, saved results in simulations/grid80_price3000_interval15.csv
executed strategies, save

# 3. analysis

In [4]:
import pandas as pd

def analyze_profit(file_path):
    # read transaction records
    df = pd.read_csv(file_path)

    # initialize wkly profit
    weekly_profit = []  

    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df.set_index('timestamp', inplace=True)

    # group by week
    weekly_data = df.resample('W').last()  # data at end time in each week

    # calc profit
    for date, week_row in weekly_data.iterrows():
        week_price = week_row['close']
        position = week_row['Position']
        money = week_row['money']

        # current crypto value
        profit = money + position * week_price
        weekly_profit.append([date, profit])

    # create df
    df_profit = pd.DataFrame(weekly_profit, columns=['Week', 'Profit'])

    output_folder = "analyses"
    if not os.path.exists("analyses"):
        os.makedirs(output_folder)
        
    # output
    file_path = os.path.basename(file_path)
    df_profit.to_csv(f'{output_folder}/analyse_{file_path}.csv', index=False)

    return profit


# output in csv
# df_profit = analyze_profit('simulation.csv')


import os

def iterate_simulation_files(folder_path):
    """
    search all csv and choose the highest-profit

    parameters:
        folder_path (str): input folder path
    return:
        None
    """
    # if folder path exists
    if not os.path.exists(folder_path):
        print(f"folder {folder_path} doesn't exist")
        return

    max_profit = -float('inf')
    best_file = None
    best_file_profit = None

    # search
    for file_name in os.listdir(folder_path):
        
        # check if fild name ends with .csv
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            profit = analyze_profit(file_path)
            print(f" {file_name} : {profit}")
            
            # update highest-profit file
            if profit is not None and profit > max_profit:
                max_profit = profit
                best_file = file_path
                best_file_profit = profit

    # choose the highest-profit file and info
    if best_file:
        print(f"file with highest-profit: {best_file}")
        print(f"total profit: {best_file_profit}")
    else:
        print("didn't find any")


# search 'simulations' folder
iterate_simulation_files('simulations')



 grid120_price2400_interval15.csv : -330406.2218840271
 grid80_price3300_interval3.csv : -45385039.19338204
 grid120_price2400_interval3.csv : -1686974.0479161516
 grid100_price3000_interval3.csv : -6598808.4621331245
 grid100_price3000_interval1.csv : -19751423.621732682
 grid120_price2400_interval1.csv : -5048650.747073352
 grid80_price3300_interval1.csv : -136174214.927971
 grid120_price3000_interval15.csv : -1428065.7750180382
 grid80_price3300_interval5.csv : -27154302.81632778
 grid100_price3000_interval5.csv : -3910887.7113986462
 grid80_price2400_interval15.csv : -319398.17878402025
 grid120_price2400_interval5.csv : -989896.6247950606
 grid80_price3000_interval15.csv : -1710253.4087797813
 grid100_price2400_interval15.csv : -328495.709148027
 grid80_price2400_interval5.csv : -956020.928706035
 grid100_price2700_interval5.csv : 924748.4201904461
 grid120_price3300_interval5.csv : -27321372.903712317
 grid100_price3000_interval15.csv : -1306971.687553931
 grid100_price2700_inter

## 3.1 compare profits in one merged table

In [7]:
import pandas as pd
import os
import re

def get_strategy(file_path):
    """
    parameters:
        file_path (str): file path

    return:
        List[int]: num list in file names
    """
    file_name = os.path.basename(file_path)

    numbers = re.findall(r'\d+', file_name)

    number_list = [int(num) for num in numbers]

    return number_list


def analyze_profit(file_path):
    # read transaction records
    df = pd.read_csv(file_path)

    # initialize wkly profit
    weekly_profit = []  

    df['timestamp'] = pd.to_datetime(df['timestamp'])  
    df.set_index('timestamp', inplace=True)

    # group by week
    weekly_data = df.resample('W').last()  # data at end time in each week

    # calc profit
    for date, week_row in weekly_data.iterrows():
        week_price = week_row['close']
        position = week_row['Position']
        money = week_row['money']

        # current crypto value
        profit = money + position * week_price
        weekly_profit.append([date, profit])

    # create df
    df_profit = pd.DataFrame(weekly_profit, columns=['Week', 'Profit'])

    output_folder = "analyses"
    if not os.path.exists("analyses"):
        os.makedirs(output_folder)
        
    # output
    file_path = os.path.basename(file_path)
    df_profit.to_csv(f'{output_folder}/analyse_{file_path}.csv', index=False)

    return profit ,df_profit

# output in csv
# df_profit = analyze_profit('simulation.csv')

import os


def iterate_simulation_files(folder_path):
    """
    search all csv and choose the highest-profit

    parameters:
        folder_path (str): input folder path
    return:
        None
    """
    # if folder path exists
    if not os.path.exists(folder_path):
        print(f"folder {folder_path} doesn't exist")
        return

    max_profit = -float('inf')  
    best_file = None
    best_file_profit = None
    strategy_profit = {}
    best_strategy = None
    df_profits = []
    
    # search
    for file_name in os.listdir(folder_path):
        
        # check if fild name ends with .csv
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            profit, df_profit = analyze_profit(file_path)
            strategy = str(get_strategy(file_path))
            df_profit = df_profit.rename(columns={"Profit":f"{str(strategy)}_Profit"})
            print(f" {file_name} : {profit}")
            strategy_profit[str(strategy)] =  profit
            df_profits.append(df_profit)
            
            # update highest-profit file
            if profit is not None and profit > max_profit:
                max_profit = profit
                best_file = file_path
                best_file_profit = profit
                best_strategy = strategy

    df = pd.concat(df_profits ,axis=1)
    df = df.loc[:, ~df.columns.duplicated()]

    df = df.rename(columns={ f"{str(best_strategy)}_Profit": f"best_{str(best_strategy)}_Profit" })
    df.to_csv("analyses/merged_profit.csv")
    
    # choose the highest-profit file and info
    if best_file:
        print(f"file with highest-profit: {best_file}")
        print(f"total profit: {best_file_profit}")
    else:
        print("didn't find any")


# search 'simulations' folder
iterate_simulation_files('simulations')



 grid120_price2400_interval15.csv : 0.0
 grid80_price3300_interval3.csv : 4905254.293555549
 grid120_price2400_interval3.csv : 0.0
 grid120_price2400_interval1.csv : 0.0
 grid80_price3300_interval1.csv : 14742225.044197554
 grid80_price3300_interval5.csv : 2959024.2470566886
 grid80_price2400_interval15.csv : 0.0
 grid120_price2400_interval5.csv : 0.0
 grid100_price2400_interval15.csv : 0.0
 grid80_price2400_interval5.csv : 0.0
 grid100_price2700_interval5.csv : 33.50363999999997
 grid120_price3300_interval5.csv : 2669355.3719449276
 grid100_price2700_interval1.csv : 17.29674749999994
 grid120_price3300_interval1.csv : 13309867.547830723
 grid80_price2400_interval1.csv : 0.0
 grid80_price2400_interval3.csv : 0.0
 grid120_price3300_interval3.csv : 4428535.365005691
 grid100_price2700_interval3.csv : 0.0
 grid120_price2700_interval5.csv : 33.50363999999997
 grid100_price3300_interval5.csv : 2630777.3980478854
 grid100_price3300_interval15.csv : 876925.023692518
 grid100_price2700_interva